# Chapter 13: Operator Overloading: Doing It Right

There are different kinds of operators in Python:

* An operator that requires two operands is a binary operator. All the binary operators use **infix** notation, which means that the operator appears between its operands. `+` and `|` are examples of **infix operators**.

* An operator that requires one operand is called a **unary operator**. `-` and `~` are examples of unary operators.

* `()` (function invocation), `.` (attribute access) and `[]` (item access/slicing) are also examples of operators.

This chapter deals mainly with infix and unary operators, and will cover:

* How Python supports infix operators with operands of different types.

* Using duck typing or explicit type checks to deal with operands of various types.

* How an infix operator method should signal it cannot handle an operand.

* The special behaviour of the rich comparison operators (e.g., `==`, `>`, `<`, `<=`, etc.).

* The default handling of augmented assignment operators, like `+=`, and how to overload them.

## Operator Overloading 101

There are some limitations imposed on operator overloading in Python. **We cannot:**

* overload the operators of built-in types.
* create new operators (only overload existing).
* overload `is`, `and`, `or` and `not`.

The rich comparison operators (e.g. `==`, `!=`, `>`, `<`, etc.) are special cases in operator overloading, which will be covered later.

## Unary Operators

*The Python Language Reference* lists three unary operators and their associated special methods:

`-` (`__neg__`): Arithmetic unary negation. If `x == -2` then `-x == 2`.

`+` (`__pos__`): Arithmetic unary plus. `x == +x`, excluding some special cases (see **p. 388**).

`~` (`__invert__`): Bitwise inverse of an integer. Defined as `~x == -(x+1)`If `x == 2` then `~x == -3`.

*The Python Language Reference* also lists `abs(...)` as a unary operator. The associated special method is `__abs__`.

The unary operators are easily implemented by defining the above special methods (requires only `self` argument).

However, remember the **fundamental rule of operators**: Always return a new object, i.e. create and return a new instance of a suitable type. **Special methods implementing unary or infix operators should never change their operands.**

The exception to this rule is `+`, where it is the best approach is (usually) to return a copy of `self`.

For `abs(...)` the result should be a scalar number.

In [ ]:
# Example 13-1. vector_v6.py: unary operators - and +

def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

def __neg__(self):
    return Vector(-x for x in self)

def __pos__(self):
    return Vector(self)

## Overloading `+` for Vector Addition

According to the [Data Model chapter](https://docs.python.org/3/reference/datamodel.html) in the official docs, sequences should support the `+` operator for concatenation and `*` operator for repetition.

In our `Vector` class we will instead implement these as mathematical vector operations, which makes more sense for the type.

We will also allow for addition of vectors of different lengths, due to practical considerations (may be used for information retrieval, etc.). To facilitate this, we use `zip_longest` and fill out the shortest `Vector` with zeros.

In [ ]:
# Example 13-4. Vector.add method, take #1

import itertools

def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fill_value=0.0)
    return Vector(a + b for a, b in pairs)

Note that this implementation allows for addition of a `Vector` with any iterable, because `zip_longest` can consume any iterable.

In [1]:
from examples.vector_v6 import Vector

v = Vector(range(4))
v + [1, 2, 3, 4, 5, 6]

Vector([1.0, 3.0, 5.0, 7.0, 5.0, ...])

However, if the operands are reversed, this fails:

In [2]:
[1, 2, 3, 4, 5, 6] + v

TypeError: can only concatenate list (not "Vector") to list

This happens because Python implements a special dispatching mechanism for the infix operator special methods. 

Given `a + b`, the interpreter performs the following steps:

1. If `a` has `__add__`, call `a.__add__(b)` and return unless it is `NotImplemented`.
2. If `a` does not have `__add__`, or calling it returns `NotImplemented`, check if `b` has `__radd__`, then call `b.__radd__(a)` and return result unless it is not implemented.
3. If `b` does not have `__radd__`, or calling it returns `NotImplemented`, raise `TypeError` with an *unsupported operand types* message.

The `__radd__` method is called the **"reflected"** or **"reversed"** version of `__add__`.

To make the mixed-type addition work in `Vector`, we must implement the `__radd__` special method. Note that this is as simple as delegating to the `__add__` method.

In [ ]:
# Example 13-4. Vector.__add__ and .__radd__ methods

import itertools

def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fill_value=0.0)
    return Vector(a + b for a, b in pairs)

def __radd__(self, other):
    return self + other

The above methods work with any iterable with numeric items, but fails when provided with a non-iterable object or iterables with non-numeric items:

In [4]:
v + 1

TypeError: zip_longest argument #2 must support iteration

In [5]:
v + "ABC"

TypeError: unsupported operand type(s) for +: 'float' and 'str'

If an operator cannot return a valid result because of type incompatibility, it should return `NotImplemented` instead of raising a `TypeError`. 

By returning `NotImplemented`, you leave the door open for the implementer of the other operand type to perform the operation when Python tries the reversed method call.

**In the spirit of duck typing, we do not test the input type of `other`, and instead catch exceptions and return `NotImplemented`.**

If the reverse method call returns `NotImplemented`, then Python will raise `TypeError` with a standard error message.

In [6]:
# Example 13-10. vector_v6_final.py: final operator + methods

def __add__(self, other):
    try:
        pairs = itertools.zip_longest(self, other, fill_value=0.0)
        return Vector(a + b for a, b in pairs)
    except TypeError:
        raise NotImplemented
        
def __radd__(self, other):
    return self + other

**Takeaway:** If an infix operator raises an exception, it aborts the operator dispatch algorithm. In the particular case of `TypeError`, it is often better to catch it and return `NotImplemented`. This allows the interpreter to  try calling the reversed operator method, which may correctly handle the computation with the swapped operands, if the are of different types.

## Overloading `*` for Scalar Multiplication

TODO